# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
from sqlalchemy import create_engine

import re

import numpy as np
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier


import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split


from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score


import pickle

In [ ]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///project_database.db')
    df = pd.read_sql_table('df_1', con = engine) 
    X = df.message.values
    Y = df.iloc[:, range(4,df.shape[1])]
    Y = Y.loc[:, Y.nunique() > 1]
    return X,Y

In [ ]:
X,Y = load_data()

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline(steps = [('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()),
                        ('mclf', MultiOutputClassifier( LogisticRegression() 
                                                          )) ] )

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42)


In [ ]:
model = pipeline.fit(X_train, y_train)

In [ ]:
Y['transport'].value_counts()

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def evaluate_model(model, X_test, y_test, col_names):

    y_pred = model.predict(X_test)
    y_test = np.array(y_test)

    metrics = []

    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(y_pred[:, i], y_test[:, i])
        precision = precision_score(y_pred[:, i], y_test[:, i], average='micro')
        recall = recall_score(y_pred[:, i], y_test[:, i], average='micro')
        f1 = f1_score(y_pred[:, i], y_test[:, i], average='micro')

        metrics.append([accuracy, precision, recall, f1])

    return pd.DataFrame(data=np.array(metrics),
                        index=col_names,
                        columns=['Accuracy', 'Precision', 'Recall', 'F1'])

In [ ]:
evaluate_model(model, X_test, y_test, y_test.columns)

In [ ]:
evaluate_model(model, X_test, y_test, y_test.columns)['F1'].describe()

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
def build_model():
    pipeline = Pipeline(steps = [('vect', CountVectorizer(tokenizer=tokenize)),
                             ('tfidf', TfidfTransformer()),
                            ('classifier', MultiOutputClassifier( LogisticRegression()
                                                                        ))])
    parameters = {
                  'classifier__estimator__penalty' : ['l1','l2'],
                  'classifier__estimator__C':[0.2, 0.6, 1, 3, 5],
                  'classifier__estimator__solver' : ['liblinear', 'saga'],
                }
    
    rs = RandomizedSearchCV(pipeline, param_distributions = parameters,
                            n_iter = 10, scoring = "accuracy", random_state = 42,
                            n_jobs = -1,  return_train_score = True, verbose = 3)

    return rs

In [ ]:
model_lr = build_model()

In [ ]:
model_lr.get_params

In [ ]:
model_lr.fit(X_train, y_train)

In [ ]:
y_pred = model_lr.predict(X_test)

In [ ]:
model_lr.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
evaluate_model(model_lr, X_test, y_test, y_test.columns)['F1'].describe()

In [ ]:
evaluate_model(model, X_test, y_test, y_test.columns)['F1'].describe()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### List of additionally applied approaches

- Added StartingVerbExtractor as a new feature

1. GridSearchCv of StartingVerbExtractor+TFIDF and RandomForest
2. RandomizedSearchCv TFIDF and RandomForest
3. RandomizedSearchCv TFIDF and XgBoost



In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = word_tokenize(sentence)

            # index pos_tags to get the first word and part of speech tag
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)
    

In [ ]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, 
                      param_grid=parameters,
                      cv = 5,
                      verbose=3)

    return cv


In [ ]:
model_rf_2 = build_model()

In [ ]:
model_rf_2.fit(X_train, y_train)

In [ ]:
def build_model2():
    pipeline = Pipeline([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier() ) )
                            ])

    parameters = {
        #'cv':[4,5,6],
        'text_pipeline__vect__ngram_range':((1, 1), (1, 2), (2,2), (1,3)),
        'text_pipeline__vect__max_df':(0.5, 0.75, 1.0),
        'text_pipeline__vect__max_features': (None, 5000, 10000, 20000),
        'text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100, 200, 500],
        'clf__estimator__min_samples_split': [2, 3, 4, 5, 6]
    }

    
    rs = RandomizedSearchCV(pipeline, param_distributions = parameters,
                            n_iter = 10, scoring = "accuracy", random_state = 42,
                            n_jobs = -1,  return_train_score = True, verbose = 3)

    return rs


In [ ]:
def build_model_xgb():
    pipeline = Pipeline([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ('clf', MultiOutputClassifier(XGBClassifier() ) )
                            ])

    parameters = {
        #'cv':[4,5,6],
        'text_pipeline__vect__ngram_range':((1, 1), (1, 2), (2,2), (1,3)),
        'text_pipeline__vect__max_df':(0.5, 0.75, 1.0),
        'text_pipeline__vect__max_features': (None, 5000, 10000, 20000),
        'text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100, 200, 500],
        'clf__estimator__min_samples_split': [2, 3, 4, 5, 6]
    }

    
    rs = RandomizedSearchCV(pipeline, param_distributions = parameters,
                            n_iter = 10, scoring = "accuracy", random_state = 42,
                            n_jobs = -1,  return_train_score = True, verbose = 3)

    return rs


In [ ]:
model_xgb = build_model_xgb()



In [ ]:
model_xgb.fit(X_train, y_train)




In [ ]:
evaluate_model(model_xgb, X_test, y_test, y_test.columns)['F1'].describe()



In [ ]:
evaluate_model(model, X_test, y_test, y_test.columns)['F1'].describe()



In [ ]:
evaluate_model(model, X_test, y_test, y_test.columns)['F1'].describe()

evaluate_model(model_lr, X_test, y_test, y_test.columns)['F1'].describe()

evaluate_model(model_xgb, X_test, y_test, y_test.columns)['F1'].describe()


### Summary of applied methods

Quick summary of applied approached
- Added StartingVerbExtractor as a new feature

0. Logistic Regression - Baseline

0. Randomized Search for Logistic Regression, F1 score is lower than Baseline

1. GridSearchCv of StartingVerbExtractor+TFIDF and RandomForest
    - StartingVerbExtractor is out of Index for many cases
    - Takes too long time to train while the best parameters provide more volatile F1 scores than Logistic regression
2. RandomizedSearchCv TFIDF and RandomForest
    - Lower F1 score than F1 score of Baseline Logistic regression
3. RandomizedSearchCv TFIDF and XgBoost
    - Lower F1 score than F1 score of Baseline Logistic regression



### 9. Export your model as a pickle file

### Since all agorithms provided more volatile and/or lower F1 score than baseline Algorithm I decided to use Baseline algorithm in web Application. Additional point to that is baseiline algorithm needs less memory, wee app will worj faster

In [ ]:
pickle.dump(model, open('model_final.pkl', 'wb'))



In [ ]:
import os
os.getcwd()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.